In [30]:
from dreal import *
import numpy as np
import torch 
x = torch.FloatTensor(3,2).uniform_(-1,1)
class Net(torch.nn.Module):
    
    def __init__(self,n_input,n_hidden,n_output,lqr,b):
        super(Net, self).__init__()
        torch.manual_seed(2)
        self.layer1 = torch.nn.Linear(n_input, n_hidden)
        self.layer2 = torch.nn.Linear(n_hidden,n_output)
        self.control = torch.nn.Linear(n_input,1,bias=True)
        self.control.weight = torch.nn.Parameter(lqr)
        self.control.bias = torch.nn.Parameter(b)
    def forward(self,x):
        sigmoid = torch.nn.Tanh()
        h_1 = sigmoid(self.layer1(x))
        out = sigmoid(self.layer2(h_1))
        u = self.control(x)
        return out,u
    
def dtanh(s):
    # Derivative of activation
    return 1.0 - s**2

lqr = torch.tensor([[-23.58639732,  -5.31421063]])
b = torch.tensor([[1.1]])
model = Net(2, 6, 1, lqr,b)

V_candidate, u = model(x)
bias = model.control.bias

# print(bias)
print(list(model.control.parameters()))


[Parameter containing:
tensor([[-23.5864,  -5.3142]], requires_grad=True), Parameter containing:
tensor([[1.1000]], requires_grad=True)]


## Learning Lyapunov function for Inverted Pendulum

In [41]:
# -*- coding: utf-8 -*-
from dreal import *
from Functions import *
import torch 
import torch.nn.functional as F
import numpy as np
import timeit 
import matplotlib.pyplot as plt

## Neural network model
Building NN with random parameters for Lyapunov function and initializing parameters of NN controller to LQR solution

LQR solution is obtained by minimizing the cost function J = ∫(xᵀQx + uᵀRu)dt, where Q is 2×2 identity matrix and R is 1×1 identity matrix

In [42]:
class Net(torch.nn.Module):
    
    def __init__(self,n_input,n_hidden,n_output,lqr,b):
        super(Net, self).__init__()
        torch.manual_seed(2)
        self.layer1 = torch.nn.Linear(n_input, n_hidden)
        self.layer2 = torch.nn.Linear(n_hidden,n_output)
        self.control = torch.nn.Linear(n_input,1,bias=True)
        self.control.weight = torch.nn.Parameter(lqr)
        self.control.bias = torch.nn.Parameter(b)
        
    def forward(self,x):
        sigmoid = torch.nn.Tanh()
        h_1 = sigmoid(self.layer1(x))
        out = sigmoid(self.layer2(h_1))
        u = self.control(x)
        return out,u

## Dynamical system

In [43]:
def f_value(x,u):
    #Dynamics
    y = []
    G = 9.81  # gravity
    L = 0.5   # length of the pole 
    m = 0.15  # ball mass
    b = 0.1   # friction
    
    for r in range(0,len(x)): 
        f = [ x[r][1], 
              (m*G*L*np.sin(x[r][0])- b*x[r][1]) / (m*L**2)]
        y.append(f) 
    #     if r==1:
    #         print(y)
 
    # print(len(y))
    y = torch.tensor(y)
    y[:,1] = y[:,1] + (u[:,0]/(m*L**2))
    
    return y

## Options

In [44]:
'''
For learning 
'''
N = 500             # sample size
D_in = 2            # input dimension
H1 = 6              # hidden dimension
D_out = 1           # output dimension

USE_CUDA = torch.cuda.is_available() 
device = torch.device('cuda:0' if USE_CUDA else 'cpu') 

torch.manual_seed(10)  
x = torch.FloatTensor(N, D_in).uniform_(-6, 6)

x_0 = torch.zeros([1, 2]) 

'''
For verifying 
'''
x1 = Variable("x1")
x2 = Variable("x2")
vars_ = [x1,x2]
G = 9.81 
l = 0.5  
m = 0.15
b = 0.1
config = Config()
config.use_polytope_in_forall = True
config.use_local_optimization = True
config.precision = 1e-2
epsilon = 0
# Checking candidate V within a ball around the origin (ball_lb ≤ sqrt(∑xᵢ²) ≤ ball_ub)
ball_lb = 0.5
ball_ub = 6

## Learning and Falsification

In [45]:
out_iters = 0
valid = False
#x = torch.from_numpy(x).float()
x = x.float()
while out_iters < 2 and not valid: 
    start = timeit.default_timer()
    lqr = torch.tensor([[-23.58639732,  -5.31421063]])     # lqr solution
    b0 = torch.tensor([[1.1]])
    model = Net(D_in,H1, D_out,lqr,b0)
    
    L = []
    i = 0 
    t = 0
    max_iters = 2000
    learning_rate = 0.01
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    
    while i < max_iters and not valid: 
        
        V_candidate, u = model(x)

        #print(V_candidate.size())
        #print(u.size())
        #dd
        
        X0,u0 = model(x_0)

        f = f_value(x,u)
        #print(len(f.t())) # transpose
        #print(len(f))
        Circle_Tuning = Tune(x)

        # Compute lie derivative of V : L_V = ∑∂V/∂xᵢ*fᵢ
        L_V = torch.diagonal(torch.mm(torch.mm(torch.mm(dtanh(V_candidate),model.layer2.weight)\
                            *dtanh(torch.tanh(torch.mm(x,model.layer1.weight.t())+model.layer1.bias)),model.layer1.weight),f.t()),0) # main diagonal elements
   
        # torch.mm mtx 곱
        # def dtanh(x):
        #    return 1-tanh(x)**2

        # With tuning term 
        Lyapunov_risk = (F.relu(-V_candidate)+ 1.5*F.relu(L_V+0.5)).mean()\
                    +2.2*((Circle_Tuning-6*V_candidate).pow(2)).mean()+(X0).pow(2) 
        # Without tuning term
#         Lyapunov_risk = (F.relu(-V_candidate)+ 1.5*F.relu(L_V+0.5)).mean()+ 1.2*(X0).pow(2)
        
        
        print(i, "Lyapunov Risk=",Lyapunov_risk.item()) 
        L.append(Lyapunov_risk.item())
        optimizer.zero_grad()
        Lyapunov_risk.backward()
        optimizer.step() 

        w1 = model.layer1.weight.data.numpy()
        w2 = model.layer2.weight.data.numpy()
        b1 = model.layer1.bias.data.numpy()
        b2 = model.layer2.bias.data.numpy()
        q = model.control.weight.data.numpy()

        # Falsification
        if i % 10 == 0:
            u_NN = (q.item(0)*x1 + q.item(1)*x2) 
            f = [ x2,
                 (m*G*l*sin(x1) + u_NN - b*x2) /(m*l**2)]
            
            # Candidate V
            z1 = np.dot(vars_,w1.T)+b1
            # print(z1)
            # dd
            a1 = []
            for j in range(0,len(z1)):
                a1.append(tanh(z1[j]))
            z2 = np.dot(a1,w2.T)+b2
            V_learn = tanh(z2.item(0))
 
            print('===========Verifying==========')        
            start_ = timeit.default_timer() 
            result= CheckLyapunov(vars_, f, V_learn, ball_lb, ball_ub, config,epsilon)
            stop_ = timeit.default_timer() 

            if (result): 
                print("Not a Lyapunov function. Found counterexample: ")
                print(result)
                x = AddCounterexamples(x,result,10)
                x = x.float()

                    
                
                # if len(result) > 1:
                #     print(x)
                #     dd
            else:  
                valid = True
                print("Satisfy conditions!!")
                print(V_learn, " is a Lyapunov function.")
            t += (stop_ - start_)
            print('==============================') 

        i += 1

    stop = timeit.default_timer()


    np.savetxt("w1.txt", model.layer1.weight.data, fmt="%s")
    np.savetxt("w2.txt", model.layer2.weight.data, fmt="%s")
    np.savetxt("b1.txt", model.layer1.bias.data, fmt="%s")
    np.savetxt("b2.txt", model.layer2.bias.data, fmt="%s")
    np.savetxt("q.txt", model.control.weight.data, fmt="%s")

    print('\n')
    print("Total time: ", stop - start)
    print("Verified time: ", t)
    
    out_iters+=1

0 Lyapunov Risk= 72.92294311523438
===========Verifying==========
Not a Lyapunov function. Found counterexample: 
x1 : [-5.6079814006448672, -5.6073700726100384]
x2 : [0.97427857925749373, 0.97459572723251242]
1 Lyapunov Risk= 60.02690887451172
2 Lyapunov Risk= 53.48793029785156
3 Lyapunov Risk= 49.25953674316406
4 Lyapunov Risk= 44.85493087768555
5 Lyapunov Risk= 40.770782470703125
6 Lyapunov Risk= 37.156402587890625
7 Lyapunov Risk= 33.96007537841797
8 Lyapunov Risk= 31.198087692260742
9 Lyapunov Risk= 28.778976440429688
10 Lyapunov Risk= 26.650840759277344
===========Verifying==========
Not a Lyapunov function. Found counterexample: 
x1 : [-0.81889067211311517, -0.80889067211311516]
x2 : [4.213750000000001, 4.2237500000000008]
11 Lyapunov Risk= 24.632694244384766
12 Lyapunov Risk= 23.31528663635254
13 Lyapunov Risk= 22.185224533081055
14 Lyapunov Risk= 20.83565902709961
15 Lyapunov Risk= 19.336763381958008
16 Lyapunov Risk= 18.039735794067383
17 Lyapunov Risk= 17.298614501953125
18 

142 Lyapunov Risk= 7.6942338943481445
143 Lyapunov Risk= 7.6805219650268555
144 Lyapunov Risk= 7.665976047515869
145 Lyapunov Risk= 7.65066385269165
146 Lyapunov Risk= 7.633875846862793
147 Lyapunov Risk= 7.616373062133789
148 Lyapunov Risk= 7.598208427429199
149 Lyapunov Risk= 7.579527854919434
150 Lyapunov Risk= 7.568955421447754
===========Verifying==========
Not a Lyapunov function. Found counterexample: 
x1 : [-1.1718750000000002, -1.1636289782906317]
x2 : [4.5347288404268271, 4.5447288404268269]
151 Lyapunov Risk= 7.47273063659668
152 Lyapunov Risk= 7.452212810516357
153 Lyapunov Risk= 7.437991619110107
154 Lyapunov Risk= 7.422871112823486
155 Lyapunov Risk= 7.405850410461426
156 Lyapunov Risk= 7.386989593505859
157 Lyapunov Risk= 7.367050647735596
158 Lyapunov Risk= 7.354616641998291
159 Lyapunov Risk= 7.335221290588379
160 Lyapunov Risk= 7.3148627281188965
===========Verifying==========
Not a Lyapunov function. Found counterexample: 
x1 : [-1.0622035742761748, -1.05330040512614

287 Lyapunov Risk= 4.958018779754639
288 Lyapunov Risk= 4.933976173400879
289 Lyapunov Risk= 4.9111809730529785
290 Lyapunov Risk= 4.890449523925781
===========Verifying==========
Not a Lyapunov function. Found counterexample: 
x1 : [0.099609375000000028, 0.10546875000000003]
x2 : [-0.49808074699623295, -0.49341526110598699]
291 Lyapunov Risk= 4.872017860412598
292 Lyapunov Risk= 4.85037899017334
293 Lyapunov Risk= 4.828118801116943
294 Lyapunov Risk= 4.80643892288208
295 Lyapunov Risk= 4.787142276763916
296 Lyapunov Risk= 4.765399932861328
297 Lyapunov Risk= 4.745596408843994
298 Lyapunov Risk= 4.724661350250244
299 Lyapunov Risk= 4.704607963562012
300 Lyapunov Risk= 4.684747219085693
===========Verifying==========
Not a Lyapunov function. Found counterexample: 
x1 : [-2.2514648437500004, -2.2500000000000004]
x2 : [4.404737509655563, 4.4058677941825808]
301 Lyapunov Risk= 4.62315034866333
302 Lyapunov Risk= 4.604857921600342
303 Lyapunov Risk= 4.586144924163818
304 Lyapunov Risk= 4.56

430 Lyapunov Risk= 3.5469186305999756
===========Verifying==========
Not a Lyapunov function. Found counterexample: 
x1 : [-0.11747227590319273, -0.11476671366386978]
x2 : [0.52522985744684436, 0.52901225732051749]
431 Lyapunov Risk= 3.521158456802368
432 Lyapunov Risk= 3.516848564147949
433 Lyapunov Risk= 3.5116655826568604
434 Lyapunov Risk= 3.5089240074157715
435 Lyapunov Risk= 3.503589153289795
436 Lyapunov Risk= 3.501984119415283
437 Lyapunov Risk= 3.498412847518921
438 Lyapunov Risk= 3.493044853210449
439 Lyapunov Risk= 3.4914286136627197
440 Lyapunov Risk= 3.48736310005188
===========Verifying==========
Not a Lyapunov function. Found counterexample: 
x1 : [-0.083885531965642746, -0.075236623202357009]
x2 : [0.49291299184221249, 0.49781558890308847]
441 Lyapunov Risk= 3.460441827774048
442 Lyapunov Risk= 3.4584412574768066
443 Lyapunov Risk= 3.4535908699035645
444 Lyapunov Risk= 3.449523448944092
445 Lyapunov Risk= 3.445539951324463
446 Lyapunov Risk= 3.443411111831665
447 Lyapun

571 Lyapunov Risk= 2.878004789352417
572 Lyapunov Risk= 2.8755762577056885
573 Lyapunov Risk= 2.87408447265625
574 Lyapunov Risk= 2.8744683265686035
575 Lyapunov Risk= 2.873398542404175
576 Lyapunov Risk= 2.87038254737854
577 Lyapunov Risk= 2.8716049194335938
578 Lyapunov Risk= 2.8718645572662354
579 Lyapunov Risk= 2.8694348335266113
580 Lyapunov Risk= 2.8681647777557373
===========Verifying==========
Not a Lyapunov function. Found counterexample: 
x1 : [-0.063714823259116635, -0.05371482325911664]
x2 : [0.52372789404697451, 0.53372789404697452]
581 Lyapunov Risk= 2.8431482315063477
582 Lyapunov Risk= 2.8421132564544678
583 Lyapunov Risk= 2.839258909225464
584 Lyapunov Risk= 2.839843273162842
585 Lyapunov Risk= 2.8398914337158203
586 Lyapunov Risk= 2.8375189304351807
587 Lyapunov Risk= 2.835864543914795
588 Lyapunov Risk= 2.8362576961517334
589 Lyapunov Risk= 2.83463716506958
590 Lyapunov Risk= 2.8333284854888916
===========Verifying==========
Not a Lyapunov function. Found counterexam

...

1371 Lyapunov Risk= 0.9443376660346985
1372 Lyapunov Risk= 0.9441993236541748
1373 Lyapunov Risk= 0.9440152645111084
1374 Lyapunov Risk= 0.9438776969909668
1375 Lyapunov Risk= 0.9436746835708618
1376 Lyapunov Risk= 0.9435230493545532
1377 Lyapunov Risk= 0.9433323740959167
1378 Lyapunov Risk= 0.9431589841842651
1379 Lyapunov Risk= 0.9429991841316223
1380 Lyapunov Risk= 0.9428462982177734
===========Verifying==========
Not a Lyapunov function. Found counterexample: 
x1 : [-0.1029822076484701, -0.093750000000000014]
x2 : [0.48927974095383031, 0.49432373963640241]
1381 Lyapunov Risk= 0.9403191804885864
1382 Lyapunov Risk= 0.9401421546936035
1383 Lyapunov Risk= 0.9399976134300232
1384 Lyapunov Risk= 0.9397943019866943
1385 Lyapunov Risk= 0.9395963549613953
1386 Lyapunov Risk= 0.9393921494483948
1387 Lyapunov Risk= 0.9392144680023193
1388 Lyapunov Risk= 0.93905109167099
1389 Lyapunov Risk= 0.9388265013694763
1390 Lyapunov Risk= 0.9386462569236755
===========Verifying==========
Not a Lyapunov

1511 Lyapunov Risk= 0.8869372010231018
1512 Lyapunov Risk= 0.8867685794830322
1513 Lyapunov Risk= 0.8865306973457336
1514 Lyapunov Risk= 0.8862959146499634
1515 Lyapunov Risk= 0.8860475420951843
1516 Lyapunov Risk= 0.8858238458633423
1517 Lyapunov Risk= 0.8856740593910217
1518 Lyapunov Risk= 0.8854480981826782
1519 Lyapunov Risk= 0.8852009177207947
1520 Lyapunov Risk= 0.8850050568580627
===========Verifying==========
Not a Lyapunov function. Found counterexample: 
x1 : [-1.6740871697005502, -1.6737116734139816]
x2 : [5.4576047349100989, 5.4582906418972357]
1521 Lyapunov Risk= 0.883759617805481
1522 Lyapunov Risk= 0.883648693561554
1523 Lyapunov Risk= 0.8834258317947388
1524 Lyapunov Risk= 0.8830773234367371
1525 Lyapunov Risk= 0.8829639554023743
1526 Lyapunov Risk= 0.882712185382843
1527 Lyapunov Risk= 0.8825326561927795
1528 Lyapunov Risk= 0.8823027610778809
1529 Lyapunov Risk= 0.8821679949760437
1530 Lyapunov Risk= 0.881757915019989
===========Verifying==========
Not a Lyapunov funct

1651 Lyapunov Risk= 0.8134939074516296
1652 Lyapunov Risk= 0.813011109828949
1653 Lyapunov Risk= 0.8124217391014099
1654 Lyapunov Risk= 0.8119564652442932
1655 Lyapunov Risk= 0.8113918304443359
1656 Lyapunov Risk= 0.8108109831809998
1657 Lyapunov Risk= 0.8104144334793091
1658 Lyapunov Risk= 0.8098913431167603
1659 Lyapunov Risk= 0.8092144131660461
1660 Lyapunov Risk= 0.8086636662483215
===========Verifying==========
Not a Lyapunov function. Found counterexample: 
x1 : [-0.10546875000000003, -0.099609375000000028]
x2 : [0.49341526110598699, 0.49808074699623295]
1661 Lyapunov Risk= 0.8061882257461548
1662 Lyapunov Risk= 0.8056107759475708
1663 Lyapunov Risk= 0.8051165342330933
1664 Lyapunov Risk= 0.8045921921730042
1665 Lyapunov Risk= 0.8040880560874939
1666 Lyapunov Risk= 0.803494393825531
1667 Lyapunov Risk= 0.8028952479362488
1668 Lyapunov Risk= 0.8023289442062378
1669 Lyapunov Risk= 0.8018403053283691
1670 Lyapunov Risk= 0.801139771938324
===========Verifying==========
Not a Lyapunov

1791 Lyapunov Risk= 0.7583470344543457
1792 Lyapunov Risk= 0.7579399347305298
1793 Lyapunov Risk= 0.7575672268867493
1794 Lyapunov Risk= 0.7571042776107788
1795 Lyapunov Risk= 0.7565364837646484
1796 Lyapunov Risk= 0.75609290599823
1797 Lyapunov Risk= 0.7556864619255066
1798 Lyapunov Risk= 0.7552277445793152
1799 Lyapunov Risk= 0.7547372579574585
1800 Lyapunov Risk= 0.7542487382888794
===========Verifying==========
Not a Lyapunov function. Found counterexample: 
x1 : [-0.10546875000000003, -0.099609375000000028]
x2 : [0.49341526110598699, 0.49808074699623295]
1801 Lyapunov Risk= 0.7521003484725952
1802 Lyapunov Risk= 0.7517215013504028
1803 Lyapunov Risk= 0.751266360282898
1804 Lyapunov Risk= 0.7507990598678589
1805 Lyapunov Risk= 0.7503741979598999
1806 Lyapunov Risk= 0.7498956322669983
1807 Lyapunov Risk= 0.7494869828224182
1808 Lyapunov Risk= 0.7490791082382202
1809 Lyapunov Risk= 0.7488029599189758
1810 Lyapunov Risk= 0.748406171798706
===========Verifying==========
Not a Lyapunov 

1931 Lyapunov Risk= 0.8845149874687195
1932 Lyapunov Risk= 0.8936595320701599
1933 Lyapunov Risk= 0.8868954181671143
1934 Lyapunov Risk= 0.8934820890426636
1935 Lyapunov Risk= 0.8878817558288574
1936 Lyapunov Risk= 0.887140154838562
1937 Lyapunov Risk= 0.886197566986084
1938 Lyapunov Risk= 0.8892852067947388
1939 Lyapunov Risk= 0.8870167136192322
1940 Lyapunov Risk= 0.8893153667449951
===========Verifying==========
Not a Lyapunov function. Found counterexample: 
x1 : [-0.10546875000000003, -0.099609375000000028]
x2 : [0.49341526110598699, 0.49808074699623295]
1941 Lyapunov Risk= 0.8866540789604187
1942 Lyapunov Risk= 0.883769690990448
1943 Lyapunov Risk= 0.8855448961257935
1944 Lyapunov Risk= 0.8806644678115845
1945 Lyapunov Risk= 0.8849924802780151
1946 Lyapunov Risk= 0.880255401134491
1947 Lyapunov Risk= 0.8838157653808594
1948 Lyapunov Risk= 0.8793494701385498
1949 Lyapunov Risk= 0.8832123279571533
1950 Lyapunov Risk= 0.8798816204071045
===========Verifying==========
Not a Lyapunov 

In [46]:
# 방금 학습 끝낸 뒤 학습된 모델 저장
PATH = './cifar_net.pth' # ./ 현재 디렉토리
torch.save(model.state_dict(), PATH)

In [47]:
# 학습된 모델 불러오기
PATH = './cifar_net.pth' # ./ 현재 디렉토리
lqr = torch.tensor([[-23.58639732,  -5.31421063]])
b0 = torch.tensor([[1.1]])
T_model = Net(D_in,H1, D_out,lqr,b0)
T_model.load_state_dict(torch.load(PATH))

<All keys matched successfully>

In [39]:
with open('q.txt', 'r') as file: 
    q_learn = file.read() 

x_init = np.array([[0.1,0.1]],dtype=np.float64)

q_learn = q_learn.split()
q_learn = [float(item) for item in q_learn]
q_learn = np.array(q_learn)
u_learn = (q_learn[0]*x_init[0][0] + q_learn[1]*x_init[0][1]) 


n = 2
K = [[q_learn[0], q_learn[1]]]
K_lqr = [[-23.58639732,  -5.31421063]]
x0 = np.array([0.3, -0.3], dtype=np.float64)
u0 = np.matmul(K,x0.T)
u0_lqr = np.matmul(K_lqr,x0.T)
dx = np.array([0,0], dtype=np.float64)

T = 0.001
tf = 3
sam = int(tf / T)
tspan = np.linspace(0, tf, sam + 1)

xs = len(tspan)
x = np.array([x0], dtype=np.float64)
x_lqr = np.array([x0], dtype=np.float64)
u_sig = np.array([u0], dtype=np.float64)
u_sig_lqr = np.array([u0_lqr], dtype=np.float64)

def plant(x, u):
    G = 9.81  # gravity
    l = 0.5   # length of the pole 
    m = 0.15  # ball mass
    b = 0.1   # friction
    dx[0] = x[1]
    dx[1] =  ((m*G*l*np.sin(x[0]) + u - b*x[1]) / (m*l**2))
    return dx

def rk5(x, u, T):
    k1=plant(x,u)*T
    k2=plant(x+k1*0.5,u)*T
    k3=plant(x+k2*0.5,u)*T
    k4=plant(x+k3,u)*T
    dx = x + ((k1+k4)/6+(k2+k3)/3)
    return dx


for i in range(0, xs - 1):
    u = np.matmul(K, x[i].T)
    u_lqr = np.matmul(K_lqr, x_lqr[i].T)

    x_next = rk5(x[i], u, T)
    x_next_lqr = rk5(x_lqr[i], u_lqr, T)
    x = np.vstack((x,x_next))
    x_lqr = np.vstack((x_lqr,x_next_lqr))
    if i >= 1:
        u_sig = np.vstack((u_sig,u))
        u_sig_lqr = np.vstack((u_sig_lqr,u_lqr))

u = np.matmul(K, x[i+1].T)
u_sig = np.vstack((u_sig,u))

u_lqr = np.matmul(K_lqr, x_lqr[i+1].T)
u_sig_lqr = np.vstack((u_sig_lqr,u_lqr))

plt.figure()
plt.plot(tspan, x[:, 0], label="x1")
plt.plot(tspan, x[:, 1], label="x2")
plt.plot(tspan, x_lqr[:, 0], ':',label="x1_lqr")
plt.plot(tspan, x_lqr[:, 1], ':',label="x2_lqr")
plt.grid()
plt.xlabel("Time")
plt.ylabel("State response")
plt.legend()
plt.show()

plt.figure()
plt.plot(tspan, u_sig[:, 0], label="u")
plt.plot(tspan, u_sig_lqr[:, 0],':', label="u_lqr")
plt.grid()
plt.xlabel("Time")
plt.ylabel("Control signal")
plt.legend()

plt.show()




FileNotFoundError: [Errno 2] No such file or directory: 'q.txt'

In [12]:

print(x)
import sys

# sys.stdout = open('stdout.txt', 'w')
# for i in range(0,x.size):
#     print(x[i][0],x[i][1])
    
# sys.stdout.close()

# ss = torch.FloatTensor([[1,2,4],[3,4,5]])
# print(ss.t())
# print(ss.size())
# print(ss.t().size())

tensor([[-0.5030, -0.2057],
        [-2.2500,  1.3803],
        [-3.4326, -1.0581],
        ...,
        [-1.8743,  4.6897],
        [-1.8749,  4.6890],
        [-1.8743,  4.6900]])


### Checking result with smaller epsilon ( Lie derivative of V <= epsilon )

In [ ]:
epsilon = -0.00001
start_ = timeit.default_timer() 
result = CheckLyapunov(vars_, f, V_learn, ball_lb, ball_ub, config, epsilon)
stop_ = timeit.default_timer() 

if (result): 
    print("Not a Lyapunov function. Found counterexample: ")
else:  
    print("Satisfy conditions with epsilon= ",epsilon)
    print(V_learn, " is a Lyapunov function.")
t += (stop_ - start_)

Satisfy conditions with epsilon=  -1e-05
tanh((0.53153824806213379 + 0.41562405228614807 * tanh((-1.1904032230377197 - 0.64355820417404175 * x1 - 0.37806212902069092 * x2)) - 0.52173459529876709 * tanh((-0.87690842151641846 - 0.082122229039669037 * x1 + 0.026046659797430038 * x2)) + 0.94497662782669067 * tanh((-0.7956465482711792 + 0.64431172609329224 * x1 + 0.092067182064056396 * x2)) - 0.80503857135772705 * tanh((1.0789084434509277 + 1.0054492950439453 * x1 + 0.01036079041659832 * x2)) + 1.1105875968933105 * tanh((1.1874630451202393 + 0.036503951996564865 * x1 + 0.0010854984866455197 * x2)) - 0.10321687906980515 * tanh((1.3883064985275269 - 0.080152533948421478 * x1 - 0.48634085059165955 * x2))))  is a Lyapunov function.


### More details on Lyapunov risk
Generally, we start training with Lyapunov risk without the tuning term.      
For example, (1* F.relu(-V_candidate)+ 1.5* F.relu(L_V+0.5)).mean()+ 1.2*(X0).pow(2)    
The weight of each term (1, 1.5, 1.2) can be tuned for balancing each Lyapunov condition.     
Furthermore, using F.relu(L_V+0.5) allows the learning procedure to seek a candidate Lyapunov function with more negative Lie derivative.   
Here 0.5 is also a tunable parameter based on your goal.    
In this example, we use Lyapunov risk with tuning term for achieving large ROA     